In [187]:
#!pip install thefuzz
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import geopandas as gpd
from unidecode import unidecode


df = pd.read_parquet("./datos/datos_servel_2022.parquet")
ee=pd.read_csv('./datos/20220914_Directorio_Oficial_EE_2022_20220430_WEB.csv',sep=';',on_bad_lines='skip')

comloc=df[['Comuna','Local']]
comee=ee[['NOM_RBD','COD_COM_RBD','NOM_COM_RBD','LATITUD','LONGITUD']]
comloc['Comuna']=comloc['Comuna'].apply(unidecode).replace('CABO DE HORNOS(EX-NAVARINO)','CABO DE HORNOS')
comloc['Local']=comloc['Local'].apply(unidecode)
comloc=comloc.drop_duplicates()
comee['NOM_COM_RBD']=comee['NOM_COM_RBD'].apply(unidecode).replace('PAIGUANO','PAIHUANO').replace('MARCHIHUE','MARCHIGUE')
comee['NOM_RBD']=comee['NOM_RBD'].astype(str).apply(unidecode)

In [189]:
union_nombres=comloc.merge(comee,left_on=['Comuna','Local'],right_on=['NOM_COM_RBD','NOM_RBD'],how='inner')

In [190]:
len(union_nombres)

1623

In [194]:
union_nombres.head()

,Comuna,Local,NOM_RBD,COD_COM_RBD,NOM_COM_RBD,LATITUD,LONGITUD
0,ALTO HOSPICIO,COLEGIO MARISTA HERMANO FERNANDO,COLEGIO MARISTA HERMANO FERNANDO,1107,ALTO HOSPICIO,"-20,282419000000000","-70,08577000000000"
1,ALTO HOSPICIO,COLEGIO SAGRADO CORAZON DE JESUS,COLEGIO SAGRADO CORAZON DE JESUS,1107,ALTO HOSPICIO,"-20,279989000000000","-70,09834300000000"
2,ALTO HOSPICIO,COLEGIO SAN ANTONIO DE MATILLA,COLEGIO SAN ANTONIO DE MATILLA,1107,ALTO HOSPICIO,"-20,268749000000000","-70,10308800000000"
3,ALTO HOSPICIO,COLEGIO SIMON BOLIVAR,COLEGIO SIMON BOLIVAR,1107,ALTO HOSPICIO,"-20,283550000000000","-70,09805299999999"
4,ALTO HOSPICIO,LICEO LOS CONDORES,LICEO LOS CONDORES,1107,ALTO HOSPICIO,"-20,255581000000000","-70,10701000000000"


In [204]:
nombres_faltantes=union_nombres.merge(comloc,right_on=['Comuna','Local'],left_on=['NOM_COM_RBD','Local'],how='outer',indicator=True,suffixes=('_foo','')).query('_merge == "right_only"')
len(nombres_faltantes)

1300

In [215]:
nombres_faltantes=nombres_faltantes[['Comuna','Local']]
nombres_faltantes

,Comuna,Local
0,ALGARROBO,LICEO TECNICO CARLOS ALESSANDRI ALTAMIRANO (2)
1,ALHUE,LICEO SARA TRONCOSO
2,ALTO BIOBIO,ESCUELA PARTICULAR CAUNICU
3,ALTO DEL CARMEN,EDUCADOR ARTURO ALVEAR RAMOS
4,ALTO DEL CARMEN,LICEO DE ALTO DEL CARMEN
...,...,...
1295,VITACURA,LICEO MARIA LUISA BOMBAL
1296,YERBAS BUENAS,ANEXO LICEO JUAN DE DIOS PUGA
1297,YERBAS BUENAS,ESCUELA DE MAITENCILLO
1298,ZAPALLAR,ESCUELA MERCEDES MATURANA GALLARDO


In [224]:
nombres_faltantes[nombres_faltantes['Comuna']=='ARICA']

,Comuna,Local
39,ARICA,ARICA COLLEGE
40,ARICA,AZAPA VALLEY SCHOOL
41,ARICA,C.P. DE ARICA
42,ARICA,CARDENAL RAUL SILVA HENRIQUEZ
43,ARICA,CENTRO DE CAPACITACION LABORAL REINO DE BELGICA
44,ARICA,COLEGIO D-91 CENTENARIO
45,ARICA,COLEGIO JUAN PABLO SEGUNDO
46,ARICA,COLEGIO MIRAMAR
47,ARICA,COMPLEJO DEPORTIVO SAUCACHE UNIVERSIDAD DE TAR...
48,ARICA,ESCUELA D-10 GENERAL JOSE MIGUEL CARRERA


In [191]:
len(comloc)

2914

In [192]:
len(comee)

16601

In [229]:
from thefuzz import process,fuzz

comunas=nombres_faltantes['Comuna'].unique()
resultado1=pd.DataFrame(columns=['NOM_RBD1','porcentaje1','id_original_servel'])
resultado2=pd.DataFrame(columns=['NOM_RBD2','porcentaje2','id_original_servel'])
resultado3=pd.DataFrame(columns=['NOM_RBD3','porcentaje3','id_original_servel'])
resultado4=pd.DataFrame(columns=['NOM_RBD4','porcentaje4','id_original_servel'])
for comuna in comunas:
    res_comuna1=nombres_faltantes.loc[nombres_faltantes['Comuna'] == comuna,]['Local'].apply(lambda x: process.extractOne(x, comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'], scorer=fuzz.partial_ratio))#[0])
    res_comuna1=pd.DataFrame(res_comuna1.tolist(),columns=['NOM_RBD1','porcentaje1','id_original_servel'], index=res_comuna1.index)
    res_comuna2=nombres_faltantes.loc[nombres_faltantes['Comuna'] == comuna,]['Local'].apply(lambda x: process.extractOne(x, comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'], scorer=fuzz.token_set_ratio))#[0])
    res_comuna2=pd.DataFrame(res_comuna2.tolist(),columns=['NOM_RBD2','porcentaje2','id_original_servel'], index=res_comuna2.index)
    res_comuna3=nombres_faltantes.loc[nombres_faltantes['Comuna'] == comuna,]['Local'].apply(lambda x: process.extractOne(x, comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'], scorer=fuzz.partial_token_sort_ratio))#[0])
    res_comuna3=pd.DataFrame(res_comuna3.tolist(),columns=['NOM_RBD3','porcentaje3','id_original_servel'], index=res_comuna3.index)
    res_comuna4=nombres_faltantes.loc[nombres_faltantes['Comuna'] == comuna,]['Local'].apply(lambda x: process.extractOne(x, comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'], scorer=fuzz.token_sort_ratio))#[0])
    res_comuna4=pd.DataFrame(res_comuna4.tolist(),columns=['NOM_RBD4','porcentaje4','id_original_servel'], index=res_comuna4.index)
    
    
    resultado1=pd.concat([resultado1,res_comuna1],axis=0,ignore_index=True)
    resultado2=pd.concat([resultado2,res_comuna2],axis=0,ignore_index=True)
    resultado3=pd.concat([resultado3,res_comuna3],axis=0,ignore_index=True)
    resultado4=pd.concat([resultado4,res_comuna4],axis=0,ignore_index=True)

In [230]:
comee2=pd.concat([nombres_faltantes,resultado1[['NOM_RBD1','porcentaje1']],resultado2[['NOM_RBD2','porcentaje2']],resultado3[['NOM_RBD3','porcentaje3']],resultado4[['NOM_RBD4','porcentaje4']]],axis=1)

In [231]:
comee2.query('porcentaje1>95')

,Comuna,Local,NOM_RBD1,porcentaje1,NOM_RBD2,porcentaje2,NOM_RBD3,porcentaje3,NOM_RBD4,porcentaje4
0,ALGARROBO,LICEO TECNICO CARLOS ALESSANDRI ALTAMIRANO (2),LICEO TECNICO CARLOS ALESSANDRI ALTAMIRANO,100,LICEO TECNICO CARLOS ALESSANDRI ALTAMIRANO,100,LICEO TECNICO CARLOS ALESSANDRI ALTAMIRANO,100,LICEO TECNICO CARLOS ALESSANDRI ALTAMIRANO,98
2,ALTO BIOBIO,ESCUELA PARTICULAR CAUNICU,ESCUELA PARTICULAR CANICU,96,ESCUELA PARTICULAR CANICU,98,ESCUELA PARTICULAR CANICU,96,ESCUELA PARTICULAR CANICU,98
3,ALTO DEL CARMEN,EDUCADOR ARTURO ALVEAR RAMOS,ESCUELA EDUCADOR ARTURO ALVEAR RAMOS,100,ESCUELA EDUCADOR ARTURO ALVEAR RAMOS,100,ESCUELA EDUCADOR ARTURO ALVEAR RAMOS,91,ESCUELA EDUCADOR ARTURO ALVEAR RAMOS,88
5,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,COLEGIO SAN ANTONIO DE MATILLA,100,COLEGIO SAN ANTONIO DE MATILLA,100,COLEGIO SAN ANTONIO DE MATILLA,90,COLEGIO SAN ANTONIO DE MATILLA,87
9,ALTO HOSPICIO,LICEO BICENTENARIO COLEGIO NIRVANA,COLEGIO NIRVANA,100,COLEGIO NIRVANA,100,COLEGIO NIRVANA,85,COLEGIO NIRVANA,61
...,...,...,...,...,...,...,...,...,...,...
1291,VITACURA,COLEGIO SAINT GEORGE'S L1,COLEGIO SAINT GEORGE S COLLEGE,96,COLEGIO SAINT GEORGE S COLLEGE,94,COLEGIO SAINT GEORGE S COLLEGE,85,COLEGIO SAINT GEORGE S COLLEGE,80
1292,VITACURA,COLEGIO SAINT GEORGE'S L2,COLEGIO SAINT GEORGE S COLLEGE,96,COLEGIO SAINT GEORGE S COLLEGE,94,COLEGIO SAINT GEORGE S COLLEGE,85,COLEGIO SAINT GEORGE S COLLEGE,80
1293,VITACURA,COLEGIO SANTA URSULA VITACURA,COLEGIO SANTA URSULA,100,COLEGIO SANTA URSULA,100,COLEGIO SANTA URSULA,100,COLEGIO SANTA URSULA,82
1296,YERBAS BUENAS,ANEXO LICEO JUAN DE DIOS PUGA,LICEO JUAN DE DIOS PUGA,100,LICEO JUAN DE DIOS PUGA,100,LICEO JUAN DE DIOS PUGA,100,LICEO JUAN DE DIOS PUGA,88


In [235]:
nombres_faltantes.iloc[967,:]

Comuna                            RENCA
Local     CENTRO EDUCACIONAL GOYENECHEA
Name: 967, dtype: object

In [ ]:
from thefuzz import process,fuzz

comunas=comee['NOM_COM_RBD'].unique()
resultado1=pd.DataFrame(columns=['NOM_RBD1','porcentaje1','id_original_servel'])
resultado2=pd.DataFrame(columns=['NOM_RBD2','porcentaje2','id_original_servel'])
resultado3=pd.DataFrame(columns=['NOM_RBD3','porcentaje3','id_original_servel'])
resultado4=pd.DataFrame(columns=['NOM_RBD4','porcentaje4','id_original_servel'])
for comuna in comunas:
    res_comuna1=comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'].apply(lambda x: process.extractOne(x, comloc.loc[comloc['Comuna'] == comuna,]['Local'], scorer=fuzz.partial_ratio))#[0])
    res_comuna1=pd.DataFrame(res_comuna1.tolist(),columns=['NOM_RBD1','porcentaje1','id_original_servel'], index=res_comuna1.index)
    res_comuna2=comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'].apply(lambda x: process.extractOne(x, comloc.loc[comloc['Comuna'] == comuna,]['Local'], scorer=fuzz.token_set_ratio))#[0])
    res_comuna2=pd.DataFrame(res_comuna2.tolist(),columns=['NOM_RBD2','porcentaje2','id_original_servel'], index=res_comuna2.index)
    res_comuna3=comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'].apply(lambda x: process.extractOne(x, comloc.loc[comloc['Comuna'] == comuna,]['Local'], scorer=fuzz.partial_token_sort_ratio))#[0])
    res_comuna3=pd.DataFrame(res_comuna3.tolist(),columns=['NOM_RBD3','porcentaje3','id_original_servel'], index=res_comuna3.index)
    res_comuna4=comee.loc[comee['NOM_COM_RBD'] == comuna,]['NOM_RBD'].apply(lambda x: process.extractOne(x, comloc.loc[comloc['Comuna'] == comuna,]['Local'], scorer=fuzz.token_sort_ratio))#[0])
    res_comuna4=pd.DataFrame(res_comuna4.tolist(),columns=['NOM_RBD4','porcentaje4','id_original_servel'], index=res_comuna4.index)
    
    
    resultado1=pd.concat([resultado1,res_comuna1],axis=0,ignore_index=True)
    resultado2=pd.concat([resultado2,res_comuna2],axis=0,ignore_index=True)
    resultado3=pd.concat([resultado3,res_comuna3],axis=0,ignore_index=True)
    resultado4=pd.concat([resultado4,res_comuna4],axis=0,ignore_index=True)

In [26]:
#df2=comloc.merge(comee,left_on=['Comuna','Local'],right_on=['NOM_COM_RBD','NOM_RBD'],how='outer')
#print(data.head())

0        None
1        None
2        None
3        None
4        None
         ... 
16029    None
16034    None
16099    None
16227    None
16537    None
Name: NOM_RBD, Length: 167, dtype: object
